In [46]:
import pandas as pd
import os
import json

In [47]:
# Create folder to save dataset
base_dir = '../../data/'
raw_dir = os.path.join(base_dir, 'raw')
curated_dir = os.path.join(base_dir, 'curated')

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

if not os.path.exists(curated_dir):
    os.makedirs(curated_dir)


subfolder = 'cost_suburb'

if not os.path.exists(os.path.join(raw_dir, subfolder)):
    os.makedirs(os.path.join(raw_dir, subfolder))

In [48]:
# read data
df = pd.read_csv(f"{curated_dir}/individual_property_preprocessed.csv")

In [49]:
# past data of median price of properties by suburb
past_data = pd.ExcelFile("../../data/raw/domain/past_data.xlsx")

print("Available sheets:", past_data.sheet_names)

Available sheets: ['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house', 'All properties']


In [50]:
# Load the postcode dictionary from the JSON file
with open('../../data/raw/suburb_to_postcodes.json', 'r') as f:
    postcode_to_suburb = json.load(f)

In [51]:
df.shape

(80496, 28)

# aggregate property by suburb to extract median_price_by_suburb_2024

In [52]:
# only need the cost data of 2024
df_2024 = df[df['Year'] == 2024]
df_2024.shape
df_2024.iloc[2]

Address                         7/7-9 Faussett Street, Albert Park VIC 3206
Cost                                                                  490.0
Property Type                                                     Apartment
Bedrooms                                                                1.0
Bathrooms                                                               1.0
Latitude                                                          -37.84167
Longitude                                                        144.955332
Closest Gov Secondary School                            Albert Park College
Age under 20                                                           20.0
Age 20-39                                                              26.0
Age 40-59                                                              29.0
Age 60+                                                                25.0
Postcode                                                             3206.0
CBD Distance

In [53]:
# Initialize an empty list to collect suburb data
suburb_data = []

# Iterate over each suburb and its list of postcodes
for suburb, postcodes in postcode_to_suburb.items():
    # Filter the domain_data for the matching postcodes
    filtered_df = df_2024[df_2024['Postcode'].isin(postcodes)]
    
    # If no properties match, skip this suburb
    if filtered_df.empty:
        continue
    
    # Initialize a dictionary to store the suburb's aggregated data
    suburb_aggregates = {}
    
    # Find the median of Cost using filtered_df
    suburb_aggregates['median_cost'] = filtered_df['Cost'].median()

    # Add the property count for this suburb
    suburb_aggregates['property_count'] = filtered_df.shape[0]
    
    # Add the suburb name to the result
    suburb_aggregates['Suburb'] = suburb
    
    # Append the suburb's data to the list
    suburb_data.append(suburb_aggregates)

# Convert the list of suburb data into a new dataframe
suburb_aggregates_df = pd.DataFrame(suburb_data)

# Show the resulting dataframe
suburb_aggregates_df.head()

# Save the DataFrame if needed
suburb_aggregates_df.to_csv(f"{raw_dir}/{subfolder}/cost_suburb_2024.csv", index=False)


# extract the past median price of each suburb

# Relative Property Price Index
Compare the price of individual property in 2024 with mean price by suburb in 2024